<a href="https://colab.research.google.com/github/right-stack/Tabnet-Paper-ammi/blob/main/Tabnet_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import pandas as pd
import matplotlib as plt
from torch.nn import Linear, ReLU, BatchNorm1d
import torch.nn.functional.glu as GLU

In [5]:
dfs = pd.read_csv("adult.data", )
#dfs.iloc[0][""]

In [6]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'earn']
dfs.columns = cols

In [7]:
dfs.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,earn
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [16]:
dfs.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32560.000000,3.256000e+04,32560.000000,32560.000000,32560.000000,32560.000000
mean,38.581634,1.897818e+05,10.080590,1077.615172,87.306511,40.437469
std,13.640642,1.055498e+05,2.572709,7385.402999,402.966116,12.347618
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178315e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783630e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370545e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [10]:
print("Workclasses: ", dfs['workclass'].unique())
print("Fnlwgt: ", dfs['fnlwgt'].unique())
print("Education: ", dfs['education'].unique())
print("Marital Status: ", dfs['marital-status'].unique())
print("Occupation: ", dfs['occupation'].unique())
print("Relationship: ", dfs['relationship'].unique())
print("Race: ", dfs['race'].unique())
print("Sex: ", dfs['sex'].unique())
print("Native Country: ", dfs['native-country'].unique())

print("Earn: ", dfs['earn'].unique())

Workclasses:  [' Self-emp-not-inc' ' Private' ' State-gov' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
Fnlwgt:  [ 83311 215646 234721 ...  34066  84661 257302]
Education:  [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
Marital Status:  [' Married-civ-spouse' ' Divorced' ' Married-spouse-absent'
 ' Never-married' ' Separated' ' Married-AF-spouse' ' Widowed']
Occupation:  [' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Adm-clerical' ' Sales' ' Craft-repair'
 ' Transport-moving' ' Farming-fishing' ' Machine-op-inspct'
 ' Tech-support' ' ?' ' Protective-serv' ' Armed-Forces'
 ' Priv-house-serv']
Relationship:  [' Husband' ' Not-in-family' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
Race:  [' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
Sex:  [' Male'

In [21]:
dfs.shape

(32560, 15)

In [22]:
len(dfs['workclass'].unique())

9

In [37]:
X = dfs[['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]

y = dfs['earn']

In [38]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States


In [39]:
y.head()

0     <=50K
1     <=50K
2     <=50K
3     <=50K
4     <=50K
Name: earn, dtype: object

In [ ]:
## LABEL ENCODING
# convert categorical features to variables 0, 1, 2 etc. 

from sklearn import preprocessing
# Reset target
le_earn = preprocessing.LabelEncoder()
le_earn.fit([' <=50K',' >50K'])   # reset earn to 0 and 1
y[:,0] = le_earn.transform(y[:,0]) 


# Transform work
le_work = preprocessing.LabelEncoder()
le_work.fit([' Self-emp-not-inc', ' Private', ' State-gov',' Federal-gov', ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay', ' Never-worked'])
X[:,1] = le_work.transform(X[:,1])

# Transform Education
le_edu = preprocessing.LabelEncoder()
le_edu.fit([' Bachelors', ' HS-grad',' 11th', ' Masters', ' 9th', ' Some-college', ' Assoc-acdm',' Assoc-voc',' 7th-8th',' Doctorate',' Prof-school', ' 5th-6th',' 10th',' 1st-4th',' Preschool',' 12th'])
X[:,3] = le_work.transform(X[:,3])



In [ ]:
# Feature Transformer class

class FeatureTransformer(nn.Module):
    def __init__(self, input_dim, output_dim):
      super(self).__init__()

      # A class implementing the feature transformer architecture as seen in the paper page 4 (https://arxiv.org/pdf/1908.07442.pdf)
      self.model = torch.nn.Sequential(*[
          # First fully connected layer
          self.fc = nn.Linear(self, input_dim, output_dim)
          # BatchNorm Layer
          self.bn = nn.BatchNorm2d(output_dim)  # BN Layer With Learnable Parameters
          # Gated linear unit (GLU) Layer
          nn.GLU()
      ])

    def forward(self, input):
      output = self.model(input)
      return output
    
    # Shared Accross Decision step
    output = self.forward(inputs)
    output1 = output
    output_part_2 = self.forward(output)
    output2 = output1 + output_part_2
    output2 = output2 / np.sqrt(0.5)

    # Decision step dependent
    output_part_3 = self.forward(output2)
    output3 = output2 + output_part_3
    output3 = output3 / np.sqrt(0.5)

    output_part_4 = self.forward(output3)
    output4 = output3 + output_part_4
    output4 = output4 / np.sqrt(0.5)


ft = FeatureTransformer()
#print(ft)

In [ ]:
# Attentive Transformer

class AttentiveTransformer(nn.Module):
    def __init__(self, self, input_dim, output_dim):
      super(self).__init__()

      # A class implementing the feature transformer architecture as seen in the paper page 4 (https://arxiv.org/pdf/1908.07442.pdf)
      
      # First fully connected layer
      self.fc = nn.Linear(input_dim, output_dim)
      # BatchNorm Layer
      self.bn = nn.BatchNorm2d(output_dim)  # BN Layer With Learnable Parameters  

    def sparseMax(inputs):
      pass  

    def forward(self, input, prior):
      output = self.fc(input)
      output = self.bn(output)
      output = torch.mul(output, prior)
      output = sparseMax(output)

      prior = output

      return output, prior

at = AttentiveTransformer()
#print(at)

In [ ]:
Class TabnetEncoder(torch.nn.Module):



## Feature transformer method
self.feature_transformer():


## Attentive transformer method
self.attentive_transformer():


## Feature Masking method
self.feature_masking():


In [ ]:
Class TabNetDecoder(torch.nn.Module):


## Feature transformer method
self.feature_transformer():

